In [196]:
%matplotlib inline
import math
import random
import sys
import matplotlib.pyplot as plt
import numpy as np
import pyNN.nest as p

In [197]:
def plot_digit(img_raw):
    img_raw = np.uint8(img_raw)
    plt.figure(figsize=(5,5))
    plt.imshow(np.reshape(img_raw,(28,28)))

In [198]:
def get_train_data():
    file_name = 'train-images.idx3-ubyte'
    f = open(file_name, "rb")
    magic_number, list_size, image_hight, image_width  = np.fromfile(f, dtype='>i4', count=4)
    train_x = np.fromfile(f, dtype='>u1', count=list_size*image_hight*image_width)
    train_x = np.reshape(train_x, (list_size,image_hight*image_width))
    f.close()
    
    file_name = 'train-labels.idx1-ubyte'
    f = open(file_name, "rb")
    magic_number, list_size = np.fromfile(f, dtype='>i4', count=2)
    train_y = np.fromfile(f, dtype='>u1', count=list_size*image_hight*image_width)
    f.close()
    
    return np.double(train_x), np.double(train_y)

In [199]:
def get_test_data():
    file_name = 't10k-images.idx3-ubyte'
    f = open(file_name, "rb")
    magic_number, list_size, image_hight, image_width  = np.fromfile(f, dtype='>i4', count=4)
    test_x = np.fromfile(f, dtype='>u1', count=list_size*image_hight*image_width)
    test_x = np.reshape(test_x, (list_size,image_hight*image_width))
    f.close()
    
    file_name = 't10k-labels.idx1-ubyte'
    f = open(file_name, "rb")
    magic_number, list_size = np.fromfile(f, dtype='>i4', count=2)
    test_y = np.fromfile(f, dtype='>u1', count=list_size*image_hight*image_width)
    f.close()
    
    return np.double(test_x), np.double(test_y)

In [200]:
def nextTime(rateParameter):
    return -math.log(1.0 - random.random()) / rateParameter
def poisson_generator(rate, t_start, t_stop):
    poisson_train = []
    if rate > 0:
        next_isi = nextTime(rate)*1000.
        last_time = next_isi
        while last_time < t_stop:
            poisson_train.append(last_time)
            next_isi = nextTime(rate)*1000.
            last_time += next_isi
    return poisson_train


In [336]:
def mnist_poisson_gen(image_list, image_height, image_width, max_freq, duration, silence):
    if max_freq > 0:
        for i in range(image_list.shape[0]):
            image_list[i] = image_list[i]/sum(image_list[i])*max_freq
    
    spike_source_data = x = [[] for i in range(image_height*image_width)]
    
    for i in range(image_list.shape[0]):
        t_start = i*(duration+silence)
        t_stop = t_start+duration
        for j in range(image_height*image_width):
            spikes = poisson_generator(image_list[i][j], t_start, t_stop)
            spike_source_data[j].extend(spikes)
            
    return spike_source_data

In [371]:
def spike_to_aerfile(spike_source_array, file_name):
    time_stamp = []
    neuron_id = []
    num_spike = len(spike_source_array)
    for i in range(num_spike):
        spikes = spike_source_array[i]
        #print spikes
        if spikes != []:
            time_stamp.extend(spikes)
            neuron_id.extend([i]*len(spikes))
            
    sort_index = sorted(range(len(time_stamp)), key=time_stamp.__getitem__)
    AllTs = np.uint32(np.ceil(np.array(time_stamp)[sort_index]*1000.)) #in mus
    neuron_id = np.array(neuron_id)[sort_index]
    y = neuron_id/image_size
    x = neuron_id%image_size
    
    AllAddr = np.uint32((x << 1) + (y << 1) * 128 + 1)
    print neuron_id, x, y,AllAddr
    f = open('abc.aedat','w')
    tok='#!AER-DAT'
    tok2='# This is a raw AE data file - do not edit'
    tok3='# Data format is int32 address, int32 timestamp (8 bytes total), repeated for each event'
    tok4='# Timestamps tick is 1 us'
    tok5='# created Tue Apr 29 11:36:59 CEST 2008'
    v=2.0

    f.write('%s'%tok)
    f.write('%1.1f\r\n'%v)
    f.write('%s\r\n'%tok2)
    f.write('%s\r\n'%tok3)
    f.write('%s\r\n'%tok4)
    f.write('%s\r\n'%tok5)

    bof=f.tell()
    #f.seek(-4,2)
    All = np.uint32(np.zeros((2,len(AllTs))))
    All[0] = AllAddr
    All[1] = AllTs
    All = np.reshape(np.transpose(All),(1,len(AllTs)+len(AllAddr)))[0]
    All = All.astype(dtype='>u4')
    All.tofile(f)
    f.close()

    return AllTs, AllAddr

In [372]:
 spike_source_data = mnist_poisson_gen(train_x[0:1], 28, 28, 5000, 1000, 100)
time_stamp, neuron_id = spike_to_aerfile(spike_source_data, '')

[299 273 574 ..., 243 354 655] [19 21 14 ..., 19 18 11] [10  9 20 ...,  8 12 23] [2599 2347 5149 ..., 2087 3109 5911]


In [368]:
neuron_id

array([747, 725, 633, ..., 771, 723, 409], dtype=uint32)

In [231]:
a=[1,6,3,9,4,9,7]
x << 1
#sorted(range(len(a)), key=a.__getitem__)

array([ 6,  6,  6, ..., 52, 52, 52])

In [171]:
s = poisson_generator(20,0,1000)
s == []

False

In [ ]:
a = range(9)
a= np.reshape(a,(3,3))
print a
a = [each*1.0/sum(each) for each in a]
a

In [ ]:
p.setup(timestep=1.0, min_delay=1.0, max_delay=3.0)
a = p.Population(2, p.SpikeSourceArray,{'spike_times':[1,2,3]} )
a[1].spike_times=[2.0,3.0,4.3]
print a[1].spike_times
p.end()

In [ ]:
np.array([2,3,4])

In [ ]:
import random
t=0
for i in range(1,10):
     t+=random.expovariate(15)*1000
     print t 

In [286]:
All = np.uint32(np.zeros((2,len(AllTs))))
All[0] = AllAddr
All[1] = AllTs
All = np.reshape(np.transpose(All),(1,len(AllTs)+len(AllAddr)))[0]
All[1]
#np.reshape(np.transpose(np.reshape(np.append(a,b),(2,3))),(1,6))

69

In [341]:
a = [1, 2, 3]
ind = [2, 1 ,0]
a = np.array(a)[ind]
a

array([3, 2, 1])